In [15]:
import numpy as np
import itertools

In [16]:
class ConvCode:
    #инициализация
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.empty_diagram = []
        self.input_one = {'input': None, 'output': None}
        self.input_zero = {'input': None, 'output': None}
        self.polinoms = self.gener_polynoms()
    

    #создание порождающих векторов
    def gener_polynoms(self):
        arr = []
        rep = self.m + 1 - 2
        for item in itertools.product([1,0], repeat=rep):
            if sum(item) == (self.m-2):
                arr.append(item)
                q = np.array(list(itertools.chain.from_iterable(arr))).reshape((self.m-1),len(arr))

        polynoms_mtrx = np.zeros((self.n,(self.m+1)))
        for i in range(self.n):
            polynoms_mtrx[i][0] = 1
            polynoms_mtrx[i][-1] = 1
            polynoms_mtrx[i][1:self.m] = q[i]
    
        return polynoms_mtrx
        

    #кодирование сообщения с помощью диаграммы состояний
    def ncd(self, ms):
        ncd_ms = []
        cur_peak = self.empty_diagram[0]
        for i in ms:
            i = int(i)
            if i:
                cur_peak, res = cur_peak.get('ones_list')
            else:
                cur_peak, res = cur_peak.get('zeros_list')

            ncd_ms.append(res)
        return np.concatenate(ncd_ms)
    

    #декодирование сообщения с помощью диаграммы состояний
    def dcd(self, ms):
        dcd_ms = []
        ms_trans = ms.reshape(-1, self.n)
        cur_peak = self.empty_diagram[0]
        for i in ms_trans:
            comparison_one = cur_peak.get('ones_list')[1]==i
            comparison_zero = cur_peak.get('zeros_list')[1]==i
            if comparison_one.all():
                dcd_ms.append(1)
                cur_peak = cur_peak.get('ones_list')[0]     
            elif comparison_zero.all():
                dcd_ms.append(0)
                cur_peak = cur_peak.get('zeros_list')[0]
        return np.array(dcd_ms)
        

    #создание диаграммы состояний
    def state_dgr(self):
      start_node = {
          'error': None,
          'time_error': None,
          'path': [],
          'time_path': [],
          'ones_list': [],
          'zeros_list': []
      }

      self.empty_diagram.append(start_node)

      index_node = '0' * self.m

      empty_nodes = [(index_node, start_node)]

      nodes = {index_node: start_node}

      for index, node in empty_nodes:
        index_ones = '1' + index[:-1]

        value = np.array([np.sum(i * np.array(list(index_ones + index[-1]), dtype=int)) % 2 for i in self.polinoms])

        if index_ones in nodes.keys():
          node['ones_list'] = [nodes.get(index_ones), value]
        else:
          node['ones_list'] = [{
              'error': None,
              'time_error': None,
              'path': [],
              'time_path': [],
              'ones_list': {},
              'zeros_list': {}
          }, value]
          empty_nodes.append((index_ones, node.get('ones_list')[0]))
          nodes[index_ones] = node.get('ones_list')[0]

          self.empty_diagram.append(node.get('ones_list')[0])

        index_zeros = '0' + index[:-1]

        value = np.array([np.sum(i * np.array(list(index_zeros + index[-1]), dtype=int)) % 2 for i in self.polinoms])

        if index_zeros in nodes.keys():
          node['zeros_list'] = [nodes.get(index_zeros), value]
        else:
          node['zeros_list'] = [{
              'error': None,
              'time_error': None,
              'path': [],
              'time_path': [],
              'ones_list': {},
              'zeros_list': {}
          }, value]
          empty_nodes.append((index_zeros, node.get('zeros_list')[0]))
          nodes[index_zeros] = node.get('zeros_list')[0]

          self.empty_diagram.append(node.get('zeros_list')[0])


    #алгоритм Витерби
    def dcd_v(self, mess, w_size):
          out_message = []

          message = [mess[i * self.n: i * self.n + self.n] for i in range(int(len(mess) / self.n))]

          self.empty_diagram[0]['time_error'] = 0
          self.empty_diagram[0]['error'] = self.empty_diagram[0].get('time_error')
          self.empty_diagram[0]['path'] = self.empty_diagram[0].get('time_path')
          self.empty_diagram[0]['time_error'] = None
          self.empty_diagram[0]['time_path'] = []

          for index in range(len(message)):
              for node in self.empty_diagram:

                  if node.get('error') is not None:

                      current_node, value = node.get('zeros_list')

                      new_err = int(node.get('error') + np.sum((message[index] + value) % 2))

                      if current_node.get('time_error') is not None and current_node.get('time_error') > new_err or current_node.get('time_error') is None:
                          current_node['time_error'] = new_err
                          current_node['time_path'] = [0] + node.get('path')
                      elif current_node.get('time_error') == new_err:
                          current_node['time_path'] = [current_node['time_path'][i] if current_node['time_path'][i] == [0] + node.get('path') else None for i in range(len(current_node['time_path']))]

                      current_node, value = node.get('ones_list')

                      new_err = int(node.get('error') + np.sum((message[index] + value) % 2))

                      if current_node.get('time_error') is not None and current_node.get('time_error') > new_err or current_node.get('time_error') is None:
                          current_node['time_error'] = new_err
                          current_node['time_path'] = [1] + node.get('path')
                      elif current_node.get('time_error') == new_err:
                          current_node['time_path'] = [current_node['time_path'][i] if current_node['time_path'][i] == [0] + node.get('path') else None for i in range(len(current_node['time_path']))]

              for element in self.empty_diagram:
                    element['error'] = element.get('time_error')
                    element['path'] = element.get('time_path')
                    element['time_error'] = None
                    element['time_path'] = []

              if len(self.empty_diagram[0].get('path')) == w_size:
                  minimal_error_node = [s for s in self.empty_diagram if s.get('error') is not None]
                  temporary_list = [i.get('error') for i in minimal_error_node]
                  minimal_error_node = minimal_error_node[temporary_list.index(min(temporary_list))]
                  position = minimal_error_node.get('path')[-1]
                  if position is not None:
                      out_message.append(position)
                  else:
                      out_message.append(0)

          minimal_error_node = [s for s in self.empty_diagram if s.get('error') is not None]
          temporary_list = [i.get('error') for i in minimal_error_node]
          minimal_error_node = minimal_error_node[temporary_list.index(min(temporary_list))]
          minimal_path = minimal_error_node.get('path').copy()
          minimal_path.reverse()
          out_message.extend(minimal_path)

          for element in self.empty_diagram:
              element['path'] = []
              element['time_path'] = []
              element['error'] = None
              element['time_error'] = None

          return np.array(out_message)

In [17]:
'''ЗАДАНИЕ #1'''

XX = ConvCode(2,3)
XX.state_dgr()
print(f'Пораждающие вектора\n {XX.gener_polynoms()}')

Пораждающие вектора
 [[1. 1. 0. 1.]
 [1. 0. 1. 1.]]


In [18]:
'''ЗАДАНИЕ #2'''

print(f'Ввели {np.array([1, 0, 1, 1, 0])}\nРезультат кодирования {XX.ncd( np.array([1, 0, 1, 1, 0]))}')
print(f'Результат декодирования {XX.dcd(XX.ncd( np.array([1, 0, 1, 1, 0])))}')

Ввели [1 0 1 1 0]
Результат кодирования [1. 1. 1. 0. 1. 0. 1. 0. 1. 1.]
Результат декодирования [1 0 1 1 0]


In [19]:
'''ЗАДАНИЕ #3'''

print(f'Ввели {np.array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1])}\nРезультат декодирования {XX.dcd(np.array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1]))}')
print(f'Результат кодирования {XX.ncd(XX.dcd(np.array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1])))}')

Ввели [1 1 1 0 1 0 1 0 1 1]
Результат декодирования [1 0 1 1 0]
Результат кодирования [1. 1. 1. 0. 1. 0. 1. 0. 1. 1.]


In [20]:
'''ЗАДАНИЕ #4'''

#задаём размер окна бОльшим, чем размер ожидаемого результата декодирования
print(f'Ввели {np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0])}\nРезультат декодирования (Витерби) {XX.dcd_v(np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0]), 6)}')

Ввели [1 1 0 0 0 0 0 0 0 0]
Результат декодирования (Витерби) [0 0 0 0 0]


In [21]:
'''ЗАДАНИЕ #4 (с ошибкой)'''

#задаём размер окна бОльшим, чем размер ожидаемого результата декодирования
print(f'Ввели {np.array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0])}\nРезультат декодирования (Витерби) {XX.dcd_v(np.array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0]), 6)}')

Ввели [1 1 0 0 0 0 0 0 1 0]
Результат декодирования (Витерби) [1 1 1 0 0]
